In [25]:
#@title Prepare elasticsearch client { display-mode: "form" }
!pip install -qU elasticsearch sentence-transformers==2.2.2
from elasticsearch import Elasticsearch
import pandas as pd
# Create the client instance
cloud_id = '\u003Ccloud_id>' #@param {type:"string"}
elastic_username = 'elastic' #@param {type:"string"}
elastic_password = '\u003Cpassword>' #@param {type: "string"}
client = Elasticsearch(
    cloud_id=cloud_id,
    basic_auth=(elastic_username, elastic_password)
)

def pretty_response_transform(response):
    result = []
    for hit in response['hits']['hits']:
        result.append({
            'id' : hit['_id'],
            'publication_date' : hit['_source']['publish_date'],
            'score' : hit['_score'],
            'title' : hit['_source']['title'],
            'summary' : hit['_source']['summary']
        })
    return result

[Quick Start](https://github.com/yansavitski/elasticsearch-labs/blob/notebooks-guides/colab-notebooks-examples/search/00-quick-start.ipynb) || **Keyword Quering Filtering** || [Hubrid search with RRF](https://github.com/yansavitski/elasticsearch-labs/blob/notebooks-guides/colab-notebooks-examples/search/02-hybrid-search-with-rrf.ipynb) || [ELSER](https://github.com/yansavitski/elasticsearch-labs/blob/notebooks-guides/colab-notebooks-examples/search/03-ELSER.ipynb)

# Keyword Quering Filtering

<a target="_blank" href="https://colab.research.google.com/github/elasticsearch-labs/colab-notebooks-examples/blob/notebooks-guides/search/01-keyword-querying-filtering.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This interactive notebook will introduce you to the Elasticsearch queries, using the official Elasticsearch Python client. Before getting start this section we highly recomented firstly finish [quick start](https://github.com/yansavitski/elasticsearch-labs/blob/notebooks-guides/colab-notebooks-examples/search/00-quick-start.ipynb)

## Querying
In the query context, a query clause answers the question “How well does this document match this query clause?” Besides deciding whether or not the document matches, the query clause also calculates a relevance score in the _score metadata field.

### Full text queries

The full text queries enable you to search analyzed text fields such as the body of an email. The query string is processed using the same analyzer that was applied to the field during indexing.

* **match**.
    The standard query for performing full text queries, including fuzzy matching and phrase or proximity queries.
* **multi-match**.
    The multi-field version of the match query.

### Compound query

Compound queries wrap other compound or leaf queries, either to combine their results and scores, to change their behaviour, or to switch from query to filter context.

* **boosting query**.
    Return documents which match a positive query, but reduce the score of documents which also match a negative query.

### Match query
Returns documents that `match` a provided text, number, date or boolean value. The provided text is analyzed before matching.

The `match` query is the standard query for performing a full-text search, including options for fuzzy matching.

[Read more](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html#match-query-ex-request)



In [52]:
response = client.search(index="book_index", query={
    "match": {
        "summary": {
            "query": "guide"
            }
        }
    })

pd.DataFrame.from_records(pretty_response_transform(response))

,id,publication_date,score,title,summary
0,3cXgIYkBfxlbyhU5Krfc,2019-10-29,0.704228,The Pragmatic Programmer: Your Journey to Mastery,A guide to pragmatic programming for software ...
1,3sXgIYkBfxlbyhU5Krfc,2019-05-03,0.704228,Python Crash Course,"A fast-paced, no-nonsense guide to programming..."
2,5MXgIYkBfxlbyhU5Krfd,2011-05-13,0.677165,The Clean Coder: A Code of Conduct for Profess...,A guide to professional conduct in the field o...
3,4MXgIYkBfxlbyhU5Krfc,2008-08-11,0.628835,Clean Code: A Handbook of Agile Software Craft...,"A guide to writing code that is easy to read, ..."
4,48XgIYkBfxlbyhU5Krfd,1994-10-31,0.628835,Design Patterns: Elements of Reusable Object-O...,Guide to design patterns that can be used in a...


### Multi-match query

The `multi_match` query builds on the match query to allow multi-field queries:

In [69]:
response = client.search(index="book_index", query={
    "multi_match": {
        "query": "javascript",
        "fields": ["summary", "title"]
        }
    })

pd.DataFrame.from_records(pretty_response_transform(response))

,id,publication_date,score,title,summary
0,3sXgIYkBfxlbyhU5Krfc,2019-05-03,2.4939,Python Crash Course,"A fast-paced, no-nonsense guide to programming..."


### Boosting query

Returns documents matching a `positive` query while reducing the `relevance score` of documents that also match a `negative` query.

You can use the `boosting` query to demote certain documents without excluding them from the search results.

In [63]:
response = client.search(index="book_index", query={
    "boosting": {
        "positive": {
            "term": {
                "summary": "javascript"
                }
            },
        "negative": {
            "term": {
                "summary": "introduction"
                }
            },
        "negative_boost": 0.5
        }
    })

pd.DataFrame.from_records(pretty_response_transform(response))

,id,publication_date,score,title,summary
0,5cXgIYkBfxlbyhU5Krfd,2008-05-15,1.254593,JavaScript: The Good Parts,A deep dive into the parts of JavaScript that ...
1,4cXgIYkBfxlbyhU5Krfc,2015-03-27,0.818029,You Don't Know JS: Up & Going,Introduction to JavaScript and programming as ...


### Prefix search

Returns documents that contain a specific prefix in a provided field

In [71]:
response = client.search(index="book_index", query={
    "prefix": {
        "title": {
            "value": 'java'
            }
        }
    })

pd.DataFrame.from_records(pretty_response_transform(response))

,id,publication_date,score,title,summary
0,4sXgIYkBfxlbyhU5Krfc,2018-12-04,1.0,Eloquent JavaScript,A modern introduction to programming
1,5cXgIYkBfxlbyhU5Krfd,2008-05-15,1.0,JavaScript: The Good Parts,A deep dive into the parts of JavaScript that ...


### Fuzzy search

Returns documents that contain terms similar to the search term, as measured by a Levenshtein edit distance.

An edit distance is the number of one-character changes needed to turn one term into another. These changes can include:

* Changing a character (box → fox)
* Removing a character (black → lack)
* Inserting a character (sic → sick)
* Transposing two adjacent characters (act → cat)



In [78]:
response = client.search(index="book_index", query={
    "fuzzy": {
        "title": {
            "value": 'pyvascript'
            }
        }
    })

pd.DataFrame.from_records(pretty_response_transform(response))

,id,publication_date,score,title,summary
0,4sXgIYkBfxlbyhU5Krfc,2018-12-04,1.624602,Eloquent JavaScript,A modern introduction to programming
1,5cXgIYkBfxlbyhU5Krfd,2008-05-15,1.365127,JavaScript: The Good Parts,A deep dive into the parts of JavaScript that ...
